In [58]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import mysql.connector
from time import sleep
import ipynb
from ipynb.fs.full.SqlQuery import brandAdd
from ipynb.fs.full.SqlQuery import categorieAdd
from ipynb.fs.full.SqlQuery import AddProduct
from ipynb.fs.full.SqlQuery import getCagId
from ipynb.fs.full.SqlQuery import getBrandId
from ipynb.fs.full.SqlQuery import AddProductAttribut

In [60]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)

In [61]:
def alarabia_links():
#     list=['Ordinateurs','Téléphonie','Stockage','Impressions','Son & Image', 'Electroménager ','Réseau & Sécurité','Accessoires','Bureautique','Surveillance','Maison & Loisirs',
#         'Gaming','SCOLAIRE']
    alarabia_list=[]
    web = requests.get('https://www.alarabia.com.tn/').content
    parse = BeautifulSoup(web, 'html.parser')
    head= parse.find('ul', class_='sf-menu').find_all('li')
#     print(head)
    for h in head:
        link= h.a['href']
        info= h.a.text
        dect={
                 'info':info,
                 'link':link
                 }
        alarabia_list.append(dect)
    return alarabia_list

In [62]:
def priceConv(price):
    c=''
    for p in price:
        if(p.isdigit()):
            c=c+p
    return int(c)

In [63]:
def zoom_details(link):
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    try:
        name= parse.find('div', class_='pb-center-column col-xs-12 col-sm-4').h1.text
    except:
        name= 'None'
    try:
        uid= parse.find('span', itemprop="sku")['content']
    except:
        uid='Undifined'
    try:
        brand_logo = parse.find('img', class_='man_prod').get('src')
    except:
        brand_logo ='Unknown'
    try:
        des = parse.find('div', class_ ='rte align_justify').text.strip()
    except:
        des= 'None'
    try:
        img = parse.find('span', id ='view_full_size').img['src']
    except:
        img='None'
    try:
        price= parse.find('span', id ='our_price_display').text
        price = priceConv(price)
    except:
        price=0
    try:
        stock = parse.find('span', id ='availability_value').text
    except:
        stock= 'non disponible'
    list=[]
    title, feautre ='',''
    try:
        fich_tech= parse.find('table', class_='table-data-sheet').find_all('tr')
        for f in fich_tech:
            title=f.td.text
            feautre= [t.text for t in f.find_all('td')][1]
            fich_dec={
                'label':title,
                'data':feautre
            }
            list.append(fich_dec)
    except:
        list=[]
    details={
        'name':name,
        'uid':uid,
        'brand_logo':brand_logo,
        'des':des,
        'img':img,
        'price':price,
        'stock': stock,
        'fich_tech':list
    }
    return details

In [65]:
# mycursor = mydb.cursor()
list = alarabia_links()
for l in list:
    info = l['info']
    link = l['link']
    categorieAdd(info, link, 24)
    supCatId=getCagId(info,24)
    for x in range(1, 20):
        print(link)
        web = requests.get(link)
        parse = BeautifulSoup(web.content, 'html.parser')
        card= parse.find_all('li', class_='ajax_block_product')
        for c in card:
            try:
                det_link= c.find('a', class_='product_img_link')['href']
            except:
                det_link='NO LINKS'
            if(det_link != 'NO LINKS'):
                det=zoom_details(det_link)  
                brandAdd(det['brand_logo'], det['brand_logo'], 24)
                supBrandId=getBrandId(det['brand_logo'], 24)
                prodId=AddProduct(det['name'], det_link, det['uid'], det['des'], det['price'], det['stock'], 24, supBrandId, supCatId, det['img'])
                if(det['fich_tech']):
                    AddProductAttribut(det['fich_tech'], prodId)
        break

Data Base Categorie Duplication Error

https://www.alarabia.com.tn/12-ordinateurs-tunisie
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
https://www.alarabia.com.tn/23-ordinateur-portable
https://www.alarabia.com.tn/282-pc-portable
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
https://www.alarabia.com.tn/279-pc-portable-gamer
Product Duplicate Error
https://www.alarabia.com.tn/353-mac
Product Duplicate Error
https://www.alarabia.com.tn/24-ordinateur-de-bureau
https://www.alarabia.com.tn/281-pc-de-bureau
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
https://www.alarabia.com.tn/280-ordinateur-gamer
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product